In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key="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",
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [8]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-34832")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Lead Data Engineer

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsData & AnalyticsLead Data EngineerKarnataka, IndiaBecome a Part of the NIKE, Inc. Team
NIKE, Inc. does more 

In [9]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [10]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Lead Data Engineer',
 'experience': 'Minimum 8+ years of working experience in designing and building production grade Microservices',
 'skills': ['Python',
  'Big Data Technologies - Databricks, EMR’s and Other Cloud Compute Tools',
  'AWS web Services (like EC2, RDS, Lambda, S3, LoadBalancer, ECS, EKS etc.)',
  'CI/CD pipeline',
  'Jenkins',
  'Terraform',
  'Docker',
  'Kubernetes',
  'OOP Concepts',
  'Data Structures',
  'Design Patterns',
  'Web Services',
  'Caching',
  'Scalability',
  'REST Principles',
  'Databases like Databricks SQL, Trino, MySQL, Postgresql etc.',
  'Data Governance'],
 'description': 'We are looking for a Lead Data Platform Engineer who excels in team environments and are excited about building cloud native platforms that can scale with the demand of our business. This role is part of Enterprise Architecture and Developer Experience aggressively innovates solutions to drive growth while creating and implementing tools that help make everything e

In [11]:
type(json_res)

dict

In [12]:
import pandas as pd
df = pd.read_csv('my_portfolio.csv')
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [13]:
import uuid
import chromadb

client = chromadb.PersistentClient("vectorstore")
collection = client.create_collection("portfolio")

if not collection.count():
    for _,row in df.iterrows():
        collection.add(documents=row["Techstack"],
                      metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [16]:
links = collection.query(query_texts=job["skills"],n_results=2).get('metadatas',[])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'l

In [15]:
job = json_res
job['skills']

['Python',
 'Big Data Technologies - Databricks, EMR’s and Other Cloud Compute Tools',
 'AWS web Services (like EC2, RDS, Lambda, S3, LoadBalancer, ECS, EKS etc.)',
 'CI/CD pipeline',
 'Jenkins',
 'Terraform',
 'Docker',
 'Kubernetes',
 'OOP Concepts',
 'Data Structures',
 'Design Patterns',
 'Web Services',
 'Caching',
 'Scalability',
 'REST Principles',
 'Databases like Databricks SQL, Trino, MySQL, Postgresql etc.',
 'Data Governance']

In [17]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Lead Data Engineer Services for Scalable Cloud Native Platforms

Dear Hiring Manager,

I came across your job posting for a Lead Data Engineer and was impressed by the role's focus on building cloud-native platforms that can scale with business demands. As a Business Development Executive at AtliQ, I'd like to introduce our company as a potential partner in fulfilling your needs.

AtliQ is an AI & Software Consulting company dedicated to facilitating seamless integration of business processes through automated tools. Our team of experts has empowered numerous enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency.

Our expertise in designing and building production-grade microservices aligns with your requirements. We have hands-on experience with Python, Big Data Technologies (Databricks, EMR's, and other Cloud Compute Tools), AWS Web Services (EC2, RDS, Lambda, S3, LoadBalancer, ECS, EKS, etc